# Properties

This note book covers the basics of properties, how they are used, what inputs they accept,
and some examples.

## What is a property?

Properties are used by Features to determine how they are resolved. They represent a value,
which can be of any data type. This value can be updated through a sampling rule which is
passed to the class constructor on initialization. Concretely, a Property always has a value
accessible through the `current_value` field. This value is updated whenever `update()` is
called, and the new value is determined by the sampling rule passed as an input to the
constructor.

## What is a sampling rule?

A sampling rule, just like the `current_value` field, can be of any type. When a rule is
sampled - that is, when you request it to create a new value - it runs through a series of 
checks, seen below: 

In [ ]:
def sample(self):
    if hasmethod(sampling_rule, "sample"):
        # If the ruleset itself implements a sample method,
        # call it instead.
        return sampling_rule.sample()

    elif isinstance(sampling_rule, dict):
        # If the ruleset is a dict, return a new dict with each
        # element being sampled from the original dict.
        out = {}
        for key, val in self.sampling_rule.items():
            if hasmethod(val, 'sample'):
                out[key] = val.sample()
            else:
                out[key] = val
        return out

    elif isiterable(sampling_rule):
        # If it's iterable, return the next value
        return next(sampling_rule)
        
    elif callable(sampling_rule):
        # If it's a function, call it without parameters
        return sampling_rule()

    elif (isinstance(sampling_rule, list) or
            isinstance(sampling_rule, np.ndarray) and sampling_rule.ndim == 1):
        # If it's either a list or a 1-dimensional ndarray,
        # return a random element from the list. 
        return np.random.choice(sampling_rule)

    else:
        # Else, assume it's elementary.
        return sampling_rule